In [2]:
def get_endpoints(data, segments, centers, masks, echo, save_path="", filename=""):

    closePoints=np.empty((len(segments),2))
    farPoints=np.empty((len(segments),2))

    for i, seg in enumerate(segments):
        if masks[i, echo, :, :].any():
            center1=centers[i,echo,:]

            if i==0:
                center2=centers[i+1,echo,:]
                u21=(center2-center1)/np.linalg.norm(center2-center1)
                u12=u21
            elif i==len(segments)-1:
                center3=centers[i-1,echo,:]
                u12=(center1-center3)/np.linalg.norm(center3-center1)
                u21=u12
            else:
                center2=centers[i+1,echo,:]
                center3=centers[i-1,echo,:]
                u21=(center2-center1)/np.linalg.norm(center2-center1)
                u12=(center1-center3)/np.linalg.norm(center3-center1)
            
            alpha=1
            while True: 
                closeP=np.round(center1+alpha*u21)
                y,x=closeP.astype(int)
                if masks[i, echo, x, y]==1:
                    alpha=alpha+1
                else:
#                     closeP=closeP-u21
                    break
            alpha=1
            while True: 
                farP=np.round(center1-alpha*u12)
                y,x=farP.astype(int)
                if masks[i, echo, x, y]==1:
                    alpha=alpha+1
                else:
#                     farP=farP+u12
                    break

            closePoints[i,:]=closeP
            farPoints[i,:]=farP

        else:
            closePoints[i,:]=center1
            farPoints[i,:]=center1

        
    plt.figure()
    plt.imshow(data[:,:,img_slice,echo], cmap='gray')
    plt.scatter(weightedCenters[:,echo,0],weightedCenters[:,echo,1], c='r', s=0.1)
    plt.scatter(closePoints[:,0], closePoints[:,1],marker="*", c='b', s=0.01)
    plt.scatter(farPoints[:,0], farPoints[:,1],marker="*", c='m', s=0.01)
    if save_path and filename:
        filename="endpoints-echo"+str(echo)+"-electrode"+filename+".png"
        plt.savefig(save_path+filename, dpi=dpi)
        
    return farPoints, closePoints

In [ ]:
def get_length(ind):
    indx, indy = ind
    allIdx=np.concatenate((np.reshape(indx,(-1,1)),np.reshape(indy,(-1,1))),axis=1)
    candidates = allIdx[spatial.ConvexHull(allIdx).vertices]
    # get distances between each pair of candidate points
    dist_mat = spatial.distance_matrix(candidates, candidates)

    # get indices of candidates that are furthest apart
    i, j = np.unravel_index(dist_mat.argmax(), dist_mat.shape)

#     plt.scatter(allIdx[:,1],allIdx[:,0])
#     plt.scatter(candidates[:,1],candidates[:,0], linewidths=1)
#     plt.scatter(candidates[i][1],candidates[i][0])
#     plt.scatter(candidates[j][1],candidates[j][0])

    p1=candidates[i][:]
    p2=candidates[j][:]
    length=np.linalg.norm((p1-p2))
    return p1, p2, length    

In [ ]:
def weighted_pixel_index(img, index, xind_w, xind_coef, yind_w, yind_coef):
    """
    Weigh the pixel indices with pixel intensities
    
    """
    xind_w.append(index[1]*((255-img[index[0],index[1]])/255))
    xind_coef.append((255-img[index[0],index[1]])/255)
                
    yind_w.append(index[0]*((255-img[index[0],index[1]])/255))
    yind_coef.append((255-img[index[0],index[1]])/255)
    
    return xind_w, xind_coef, yind_w, yind_coef

In [ ]:
def get_neigh_basestruct(idx, basestructs, labelsdf, segmentation_mask):
    """
    idx: index to be checked which baseline is nearest to
    basestructs: list of struct names as possible baselines
    labelsdf: df including the segmentation names and values
    segmentation_mask: corresponding segmentation img slice
    """
    dist=[]
    neighidx=[]
    for base in basestructs:
        base_seg=labelsdf["Labels"][labelsdf["Anatomical Regions"]==base].values[0]
        baseidx=np.array(list(zip(*np.where(segmentation_mask==base_seg))))
        normarr=np.linalg.norm(baseidx-idx, axis=1)
        closestidx=np.argmin(normarr)

        dist.append(normarr[closestidx])
        neighidx.append(baseidx[closestidx])
        
    struct=basestructs[np.argmin(dist)]
    base_seg=labelsdf["Labels"][labelsdf["Anatomical Regions"]==struct].values[0]
    return base_seg, struct, neighidx, dist

In [2]:
def find_roi(data, segmentation, basestructs, labelsdf, roi_name, path, path_extension="", savefigs=False):
    """
    Returns the roi which is showing significant pixel intensities compared to the baseline
    """
#     basestruct=["crtx","cc","hpc", "thlms"]
# path=root+analysed
#     roi_name="mono"
#     img_slice=2
#     segmentation_mask=coronal1seg[:,:,img_slice] ---> segmentation
    # seg_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"]==roi_namesegname].values
    
    roi_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"].str.contains(roi_name)]
    print("Segment of interest: "+roi_name+" sitk-label:"+str(roi_areas))
    save_path=path+roi_name+"-regionspecific/"+"roi"+path_extension+"/"
#     print(roi_areas)
    dpi=3000
#     savefigs=True
    zthr=-1.95

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if len(data.shape)==3:
        num_echos=data.shape[-1]
        masks=np.zeros((len(roi_areas), num_echos, segmentation.shape[0],segmentation.shape[1]))
    else:
        num_echos=0
        masks=np.zeros((len(roi_areas), segmentation.shape[0],segmentation.shape[1]))
        
    roi_px_intensities=[]
    roi_idx=get_idx2nearestNeigh(basestructs)
    roiDict={}
    roiDict=dataDict(roi_areas, basestructs, num_echos)
    print("numechos:"+str(num_echos))
    for j,seg in enumerate(roi_areas):
        tmpvals=[]
        if num_echos>0:
            for echo in range(num_echos):
                roi_echovals=[]

                img=data[:,:,echo]
                tmpidx=[]
                for index in np.array(list(zip(*np.where(segmentation==seg)))):
                    base_seg,struct,_,_=get_neigh_basestruct(index, basestructs, labelsdf, segmentation)
                    baseline=get_baseline_vals(base_seg, segmentation, data)        
                    imgz=img_zscore(img[index[0],index[1]],baseline[:,echo])
                    
                    if imgz<zthr:
#                   
                        #Grouping indeces by their neighboring baseline anatomical structure 
                        roi_idx[struct].append(index)
#                         vals=[index[0],index[1], img[index[0],index[1]]]
#                         roiDict[seg][struct][echo].append(vals)
                        roiDict[seg][struct][echo]['index'].append(index)
                        roiDict[seg][struct][echo]['intensity'].append(img[index[0],index[1]])
                        #saving the pixel intensities for given roi
                        roi_echovals.append(img[index[0],index[1]])
                        #mask of the roi
                        masks[j,echo,index[0],index[1]]=1
                
                tmpvals.append(roi_echovals)
#                 plot_roi(img, roi_idx, roi_name, seg, echo, save_path, savefigs=savefigs, dpi=dpi)
#                 plot_roi(img, roiDict, roi_name, seg, echo, save_path, savefigs=savefigs, dpi=dpi)
                
        else:
            roi_echovals=[]
            echo=0
            img=data
            for index in np.array(list(zip(*np.where(segmentation==seg)))):
                base_seg,struct,_,_=get_neigh_basestruct(index, basestructs, labelsdf, segmentation)
                baseline=get_baseline_vals(base_seg, segmentation, data)        
                imgz=img_zscore(img[index[0],index[1]],baseline)

                if imgz<zthr:
                    #Grouping indeces by their neighboring baseline anatomical structure 
                    roi_idx[struct].append(index)
                    #saving the pixel intensities for given roi
                    roi_echovals.append(img[index[0],index[1]])
                    #mask of the roi
                    masks[j,index[0],index[1]]=1

            tmpvals.append(roi_echovals)
            plot_roi(img, roi_idx, roi_name, seg, echo, save_path, savefigs=savefigs, dpi=dpi)
                
        roi_px_intensities.append(tmpvals)
        
    return roi_px_intensities, roi_idx, roiDict

In [ ]:
def get_baseline_vals(seg, segmentationArr, data):
    """
    img_s: image slice to be analysed
    seg: segmentation id of baseline
    segmentationArr: segmentation np array
    data: mri raw np array
    
    returns;
    baseline: pixel values for baseline area nparray (len(vals), len(echos))
    """
#     segmentation=segmentationArr[:,:,img_s]
    if len(data.shape)==3:
        num_echos=data.shape[-1]
    else:
        num_echos=0
    
    if num_echos>0:
        for echo in range(num_echos):
    #         img=data[:,:,img_s,echo]
            img=data[:,:,echo]
            vals=img[segmentationArr==seg]

            if echo==0:
                baseline=np.zeros((len(vals),num_echos))
                baseline[:,0]=vals
            else:
                baseline[:,echo]=vals
    else:
        baseline=data[segmentationArr==seg]
            
    return baseline

In [19]:
def get_idx2nearestNeigh(basestructs):
    structs_idx={}
    for struct in basestructs:
        structs_idx[struct]=[]
        
    return structs_idx

In [1]:
def dataDict(roi_areas, basestructs, num_echos):
    data={}
    for roi in roi_areas:
        data[roi]={}
        for struct in basestructs:
            data[roi][struct]={}
            for echo in range(num_echos):
                data[roi][struct][echo]={"index": [],
                                        "intensity":[]}
    return data